## Hydropandas

Hydropandas is a Python package for reading, writing and analyzing hydrological time series. Just like any Python package you will have to install it the first time you use it.

In [ ]:
# install the hydropandas package (you only have to do this once)
! pip install hydropandas

In [ ]:
# import all the packages required for this notebook
# if you get an import error try installing this package using pip or conda

import hydropandas as hpd
import pandas as pd
import numpy as np
from bokeh.io import show

## Groundwater Observation

You can create a GroundwaterObs object using the Hydropandas package. To do this you define a measurement time series and the metadata of the observation well. See the example below.

In [ ]:
hpd.GroundwaterObs?

In [ ]:
# Create a groundwater observation object from scratch
o = hpd.GroundwaterObs(index=pd.date_range('1-1-2023', '13-1-2023'), 
                      data=np.random.randint(0,10, size= (13,1)),
                      columns=['values'],
                      name='my observation', monitoring_well='w1',
                      screen_top=35.2, screen_bottom=32.2,
                      x=123456, y=654321, unit='MASL', tube_nr=1, 
                      metadata_available=True)
print(o)

For some Dutch hydrological databases we can use the API to automatically download the data using the ID. The example below shows how to obtain the measurement data using an ID from the BRO database.

In [ ]:
# obtain a groundwater observation from the BRO (Dutch) database
o = hpd.GroundwaterObs.from_bro('GLD000000012893')
print(o)

`o` is now a GroundwaterObs object. A GroundwaterObs object has the same attributes and methods as a pandas DataFrame. We can use these methods to do some analysis.

In [ ]:
# get some statistics of the observations
o['values'].describe()

Additionally a GroundwaterObs object has some extra attributes:
- "name"
- "x"
- "y"
- "source"
- "unit"
- "monitoring_well"
- "tube_nr"
- "screen_top"
- "screen_bottom"
- "ground_level"
- "tube_top"

We can use these attributes, for example when plotting the data. Below we use the "unit" and "name" attribute when plotting the time series.

In [ ]:
# plot the data
o['values'].plot(ylabel=o.unit, label=o.name, 
                 legend=True, figsize=(13,6), grid=True);

In [ ]:
# filter data based on the qualifier, 'goedgekeurd' means 'approved'.
o_selection = o.loc[o['qualifier']=='goedgekeurd']
# plot filtered data
o_selection.plot();

## ObsCollection

In a project you usualy you have multiple wells with measurements. In Hydropandas we can store a number of GroundwaterObs object in an ObsCollection object. We demonstrate this using the code below.

In [ ]:
# create one GroundwaterObs object
o1 = hpd.GroundwaterObs(index=pd.date_range('1-1-2023', '13-1-2023'), 
                        data=np.random.randint(0,10, size= (13,1)),
                         columns=['values'],
                        name='my observation 1', monitoring_well='w1',
                        screen_top=35.2, screen_bottom=32.2,
                        x=123456, y=654321, unit='MASL', tube_nr=1, 
                        metadata_available=True)

# create another GroundwaterObs object
o2 = hpd.GroundwaterObs(index=pd.date_range('2021-5-1', '2021-5-13'), 
                        data=np.random.randint(15,30, size= (13,1)),
                         columns=['values'],
                         name='my observation 2', monitoring_well='w2',
                        screen_top=6.7, screen_bottom=2.7,
                        x=123465, y=654312, unit='MASL', tube_nr=1, 
                        metadata_available=True)

# store both GroundwaterObs object in an ObsCollection
oc = hpd.ObsCollection(o1,o2)
oc

Just as with the GroundwaterObs object for some Dutch databases we can use the API to automatically download an ObsCollection. The example below shows how to obtain multiple observations within a certain extent from the BRO database.

In [ ]:
oc = hpd.read_bro(extent=(117850, 118180, 439550, 439900), keep_all_obs=False)
oc

You can see that all the additional attributes of a GroundwaterObs object are used as columns in the ObsCollection. The last 'obs' column contains the GroundwaterObs object itself. We can get this groundwater obs object using the usual pandas methods of indexing, slicing and subselecting. See two examples below.

In [ ]:
# get the groundwater observation with name 'GMW000000036327_1'
o = oc.loc['GMW000000036327_1','obs']
print(o)

In [ ]:
# get all the groundwater observations with a screen bottom below NAP -1.5 m
oc.loc[oc['screen_bottom'] < -1.5]

Just like a GroundwaterObs obejct an ObsCollection object has the same attributes and methods as a pandas DataFrame. Additionally an ObsCollection object has methods stored in submodules:
`geo`:
- `get_bounding_box` -> get a tuple with (xmin, ymin, xmax, ymax)
- `get_extent` -> get a tule with (xmin, xmax, ymin, ymax)
- `get_lat_lon` -> to get the lattitudes and longitudes from the x and y coordinates
- `within_polygon` -> to select only the observations that lie within a polygon

`gwobs`:
- `set_tube_nr` -> to set the tube numbers based on the tube screen depth when there are multiple tubes at one monitoring well
- `set_tube_nr_monitoring_well` -> find out which observations are at the same location with a different screen depth. Set monitoring_well and tube_nr attributes accordingly.

`plots`:
- `interactive_figures` -> create bokeh figures for each observation point.
- `interactive_map` -> create a folium map with observation points and bokeh figures for each observation point.
- `section_plot` -> create a plot of multiple observations and a plot of the well layout.

`stats`:
- `get_first_last_obs_date()` -> get the first and the last date of the observations for each observation point
- `get_no_of_observations()` -> get the number of observations
- `get_seasonal_stat()` -> get seasonal stats of the observations

In the example below we use the `set_lat_lon` method in the `geo` module to get the lattitude and longitude for our observations. Then we use the `interactive_map` method of the `plots` module to create an interactive map with graphs for each observation.

In [ ]:
oc.geo.set_lat_lon() # get lattitude and lognitude for each observation
oc.plots.interactive_map(plot_dir="figure") # create an interactive map

More information on Hydropandas can be found in the official [documentation](#https://hydropandas.readthedocs.io/en/stable/).